In [1]:
import numpy as np

import tensorflow as tf

from PIL import Image
import os

import mtcnn

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

my_image_path = "../../faces/"
dataset_image_path = "../../../Dataset/data/lfw/"
image_files = [f for f in os.listdir(my_image_path) if f[-5:] != '2.jpg']

In [2]:
tf_lite_model = tf.lite.Interpreter(model_path='../../tflite/facenet.tflite')
tf_lite_model.allocate_tensors()

input_details = tf_lite_model.get_input_details()
output_details = tf_lite_model.get_output_details()

print("== Input details ==")
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("\n== Output details ==")
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
shape: [  1 160 160   3]
type: <class 'numpy.float32'>

== Output details ==
shape: [  1 512]
type: <class 'numpy.float32'>


In [3]:
input_details[0]

{'name': 'input_1',
 'index': 427,
 'shape': array([  1, 160, 160,   3], dtype=int32),
 'shape_signature': array([  1, 160, 160,   3], dtype=int32),
 'dtype': numpy.float32,
 'quantization': (0.0, 0),
 'quantization_parameters': {'scales': array([], dtype=float32),
  'zero_points': array([], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [4]:
def get_image_embedding(image_path):
    # Open image
    image = Image.open(image_path).convert('RGB')
    pixels = np.asarray(image)
    # detect faces in the image
    detector = mtcnn.MTCNN()
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize((160, 160))
    face_array = np.asarray(image)
    face_array_norm = face_array /255
    
    tf_lite_model = tf.lite.Interpreter(model_path='../facenet.tflite')
    tf_lite_model.allocate_tensors()

    input_details = tf_lite_model.get_input_details()
    output_details = tf_lite_model.get_output_details()

    tf_lite_model.set_tensor(input_details[0]['index'], face_array_norm.reshape(1,160,160,3).astype(np.float32))

    tf_lite_model.invoke()

    return tf_lite_model.get_tensor(output_details[0]['index'])

In [9]:
embedding = get_image_embedding(dataset_image_path+'Park_Jie-won/Park_Jie-won_0001.jpg')

print([ "{:0.8f}".format(x) for x in embedding[0] ])

['0.12271542', '-0.05017735', '-0.01172269', '-0.00496438', '-0.04728256', '-0.01437640', '-0.05410565', '0.03619444', '-0.03831248', '-0.00817656', '0.03537451', '0.00239592', '-0.04064424', '-0.03707179', '0.03097228', '0.00391851', '0.06726867', '0.08617418', '0.01081356', '0.07967211', '0.02706336', '0.03636722', '-0.00501422', '-0.05927857', '-0.04349062', '-0.01967310', '-0.00893688', '0.00769436', '-0.01931625', '0.04266050', '0.03951887', '-0.03999641', '0.00309413', '0.05222530', '-0.04698726', '-0.05561155', '-0.07005093', '0.02110602', '0.08340495', '-0.03258466', '-0.00494586', '-0.07438308', '-0.01766104', '-0.02435465', '-0.03120811', '0.02593219', '-0.04660319', '-0.03907661', '-0.02785796', '0.03695910', '0.03202637', '-0.02465991', '0.05083499', '0.07203179', '0.01300457', '-0.07942011', '-0.00491865', '0.00673868', '-0.00768554', '0.02836180', '-0.06712019', '-0.00444017', '-0.00947029', '0.14830475', '-0.01495152', '-0.08807851', '0.02143637', '-0.04509759', '0.03449